In [278]:
import tensorflow as tf
import pandas as pd
from sklearn.model_selection import train_test_split

In [279]:
df = pd.read_csv('banana_dataset.csv')
print(df.head(5))

                  created_at  entry_id  Temperature  Humidity  Ethanol (Gas)  \
0  2022-06-11T15:53:46+00:00         1         34.2      81.6          425.0   
1  2022-06-11T15:54:39+00:00         2         34.3      83.8          425.0   
2  2022-06-11T15:55:22+00:00         3         34.4      84.6          429.0   
3  2022-06-11T15:55:55+00:00         4         34.4      85.7          438.0   
4  2022-06-11T15:56:28+00:00         5         34.5      85.6          424.0   

  Type of Fruit  
0        Banana  
1        Banana  
2        Banana  
3        Banana  
4        Banana  


In [280]:
def synthesize_ethylene(gas):
    ethylene_baseline = 50
    ethylene_increase_rate = 0.5
    peak_gas = 350
    decline_rate = 0.1
    
    if gas < peak_gas:
        return ethylene_baseline + (gas * ethylene_increase_rate)
    else:
        return ethylene_baseline + (peak_gas * ethylene_increase_rate) - ((gas - peak_gas) * decline_rate)


In [281]:
df['Ethylene'] = df['Ethanol (Gas)'].apply(synthesize_ethylene)
df

,created_at,entry_id,Temperature,Humidity,Ethanol (Gas),Type of Fruit,Ethylene
0,2022-06-11T15:53:46+00:00,1,34.2,81.6,425.0,Banana,217.5
1,2022-06-11T15:54:39+00:00,2,34.3,83.8,425.0,Banana,217.5
2,2022-06-11T15:55:22+00:00,3,34.4,84.6,429.0,Banana,217.1
3,2022-06-11T15:55:55+00:00,4,34.4,85.7,438.0,Banana,216.2
4,2022-06-11T15:56:28+00:00,5,34.5,85.6,424.0,Banana,217.6
...,...,...,...,...,...,...,...
2240,2022-06-29T14:27:19+00:00,2241,34.9,96.4,290.0,Banana,195.0
2241,2022-06-29T14:27:53+00:00,2242,35.1,95.8,292.0,Banana,196.0
2242,2022-06-29T14:28:26+00:00,2243,35.2,95.3,299.0,Banana,199.5
2243,2022-06-29T14:29:00+00:00,2244,35.5,94.8,312.0,Banana,206.0


In [282]:
df["Grade"] = [0 for i in range (2245)]

# Grade 1: Unripe
df.loc[(df['Ethylene'] < 130) & (df['Ethanol (Gas)'] < 130), 'Grade'] = 1

# Grade 2: Ripe (perfect to eat)
df.loc[((df['Ethylene'] >= 130) & (df['Ethylene'] < 180)) & ((df['Ethanol (Gas)'] >= 130) & (df['Ethanol (Gas)'] < 210)), 'Grade'] = 2

# Grade 3: Over ripe
df.loc[((df['Ethylene'] >= 180) & (df['Ethylene'] < 200)) & ((df['Ethanol (Gas)'] >= 210) & (df['Ethanol (Gas)'] < 360)), 'Grade'] = 3

# Grade 4: Rotting stage
df.loc[(df['Ethylene'] >= 200) | (df['Ethanol (Gas)'] >= 360), 'Grade'] = 4

In [283]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2245 entries, 0 to 2244
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   created_at     2245 non-null   object 
 1   entry_id       2245 non-null   int64  
 2   Temperature    2245 non-null   float64
 3   Humidity       2245 non-null   float64
 4   Ethanol (Gas)  2231 non-null   float64
 5   Type of Fruit  2245 non-null   object 
 6   Ethylene       2231 non-null   float64
 7   Grade          2245 non-null   int64  
dtypes: float64(4), int64(2), object(2)
memory usage: 140.4+ KB


In [284]:
df_drop = df.drop(['created_at','entry_id' ], axis=1)

In [285]:
df_new = df_drop.dropna()

In [286]:
df_new.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2231 entries, 0 to 2244
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Temperature    2231 non-null   float64
 1   Humidity       2231 non-null   float64
 2   Ethanol (Gas)  2231 non-null   float64
 3   Type of Fruit  2231 non-null   object 
 4   Ethylene       2231 non-null   float64
 5   Grade          2231 non-null   int64  
dtypes: float64(4), int64(1), object(1)
memory usage: 122.0+ KB


In [287]:
X = df_new[['Ethanol (Gas)', 'Ethylene', 'Temperature', 'Humidity']]
y = df_new['Grade'].values

In [288]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=101)
model = tf.keras.Sequential([
    tf.keras.layers.Dense(2, activation='relu', input_shape=(4,)),
    tf.keras.layers.Dense(1, activation="sigmoid", input_shape=(4,)),
    tf.keras.layers.Dense(1, activation="sigmoid", input_shape=(4,)),
    tf.keras.layers.Dense(1, activation="sigmoid", input_shape=(4,))
])

c:\Users\USER\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:86: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


ValueError: Unrecognized keyword arguments passed to Dense: {'activataion': 'relu'}

In [ ]:
model.compile(optimizer='sgd', loss="mse", metrics=['accuracy'])

In [ ]:
model.fit(X_train, y_train, epochs=50, batch_size=10, verbose=1)

loss, accuracy = model.evaluate(X_test, y_test)
print("Accuracy:", accuracy)

Epoch 1/50


157/157 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.1499 - loss: 7.9318  
Epoch 2/50
157/157 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.1458 - loss: 6.7283
Epoch 3/50
157/157 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.1411 - loss: 6.5244
Epoch 4/50
157/157 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.1549 - loss: 6.1961
Epoch 5/50
157/157 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.1521 - loss: 6.0979
Epoch 6/50
157/157 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.1554 - loss: 6.0658
Epoch 7/50
157/157 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.1519 - loss: 6.1737
Epoch 8/50
157/157 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.1577 - loss: 6.1258
Epoch 9/50
157/157 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.1454 - loss: 6.1181
Epoch 10/50
157/157 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.1507 - loss: 6.2083  
Epoch 11/50
157/157 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.1549 - loss: 6.1627
Epoch 12/50
157/157 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accu